# Temporal Graphs and Path Data

## Prerequisites

First, we need to set up our Python environment that has PyTorch, PyTorch Geometric and PathpyG installed. Depending on where you are executing this notebook, this might already be (partially) done. E.g. Google Colab has PyTorch installed by default so we only need to install the remaining dependencies. The DevContainer that is part of our GitHub Repository on the other hand already has all of the necessary dependencies installed. 

In the following, we install the packages for usage in Google Colab using Jupyter magic commands. For other environments comment in or out the commands as necessary. For more details on how to install `pathpyG` especially if you want to install it with GPU-support, we refer to our [documentation](https://www.pathpy.net/dev/getting_started/). Note that `%%capture` discards the full output of the cell to not clutter this tutorial with unnecessary installation details. If you want to print the output, you can comment `%%capture` out.

In [1]:
%%capture
# !pip install torch
!pip install torch_geometric
!pip install git+https://github.com/pathpy/pathpyG.git

## Motivation and Learning Objectives

In this tutorial we will introduce the representation of temporal graph data in the `Temporal Graph` class and how such data can be used to calculate time respecting paths.

In [1]:
import torch
from torch_geometric.data import TemporalData
from torch_geometric.utils import remove_isolated_nodes
import numpy as np
import pathpyG as pp

pp.config['torch']['device'] = 'cuda'

In [2]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.N)
print(t.M)

a -> 0
b -> 1
c -> 2
d -> 3
e -> 4
f -> 5
g -> 6
h -> 7
i -> 8

9
20


In [3]:
m = pp.MultiOrderModel.from_temporal_graph(t, max_order=3, delta=2)
print(m.layers[2])
for e in m.layers[2].edges:
    print(e)

Directed graph with 17 nodes and 3 edges

Node attributes
	node_sequence		<class 'torch.Tensor'> -> torch.Size([17, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

(('a', 'c'), ('c', 'h'))
(('a', 'g'), ('g', 'h'))
(('c', 'f'), ('f', 'a'))


In [ ]:
print(t.data)

TemporalData(src=[20], dst=[20], t=[20])


In [5]:
td = TemporalData(
    src = torch.Tensor([0,1,2,0]),
    dst = torch.Tensor([1,2,3,1]), 
    t = torch.Tensor([0,1,2,3]))
print(td)

TemporalData(src=[4], dst=[4], t=[4])


In [6]:
t2 = pp.TemporalGraph(td)

In [7]:
t1 = t.get_window(0,4)
print(t1)
print(t1.N)
print(t1.M)

Temporal Graph with 5 nodes, 4 unique edges and 4 events in [1.0, 9.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([4])
	t		<class 'torch.Tensor'> -> torch.Size([4])
	src		<class 'torch.Tensor'> -> torch.Size([4])

5
4


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'dst', 't', 'src'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [8]:
g = t.to_static_graph()
print(g)

Graph with 9 nodes and 20 edges

Graph attributes
	mapping		<class 'pathpyG.core.IndexMap.IndexMap'>
	num_nodes		<class 'int'>



In [9]:
g = t.to_static_graph((1, 10))
print(g)

Graph with 9 nodes and 17 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([17])

Graph attributes
	num_nodes		<class 'int'>



In [10]:
r = pp.algorithms.RollingTimeWindow(t, 10, 10, return_window=True)
for g, w in r:
    print('Time window ', w)
    print(g)
    print(g.data.edge_index)
    print('---')

Time window  (1.0, 11.0)
Graph with 5 nodes and 4 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([4])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 1, 2, 2],
           [1, 2, 3, 4]], sparse_size=(3, ?), nnz=4, sort_order=row)
---
Time window  (11.0, 21.0)
Graph with 7 nodes and 3 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 2, 5],
           [6, 5, 0]], sparse_size=(6, ?), nnz=3, sort_order=row)
---
Time window  (21.0, 31.0)
Graph with 8 nodes and 6 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([6])

Graph attributes
	num_nodes		<class 'int'>

EdgeIndex([[0, 0, 1, 2, 6, 7],
           [2, 6, 5, 5, 7, 5]], sparse_size=(8, ?), nnz=6, sort_order=row)
---
Time window  (31.0, 41.0)
Graph with 8 nodes and 3 edges

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([3])

Graph attributes
	num_nodes		<class 'int'>

E

## Temporal Graphs

Let's start with a simple temporal graph with four nodes `a`,`b`,`c`,`d` and seven timestamped edges `(b,c;2)`,`(a,b;1)`,`(c,d;3)`,`(d,a;4)`,`(b,d;2)`, `(d,a;6)`,`(a,b;7)`. 

The following code generates this temporal graph from the given edge list.

In [11]:
g = pp.TemporalGraph.from_edge_list([['b', 'c', 2],['a', 'b', 1], ['c', 'd', 3], ['d', 'a', 4], ['b', 'd', 2], ['d', 'a', 6], ['a', 'b', 7]])
print(g)

Temporal Graph with 4 nodes, 5 unique edges and 7 events in [1.0, 7.0]

Graph attributes
	dst		<class 'torch.Tensor'> -> torch.Size([7])
	t		<class 'torch.Tensor'> -> torch.Size([7])
	src		<class 'torch.Tensor'> -> torch.Size([7])



We can visualize a temporal graph by using the pathpyG plot function.

In [12]:
pp.plot(g, edge_color='lightgray')

Consistent with `pyG` the sources, destinations and timestamps are stored as a `pyG TemporalData` object, which we can access in the following way.



In [13]:
g.data

TemporalData(src=[7], dst=[7], t=[7])

In [14]:
print(g.data.t)

tensor([1., 2., 2., 3., 4., 6., 7.])


With the generator functions `edges` and `temporal_edges` we can iterate through the (temporal) edges of this graph.

In [15]:
for v, w in g.edges:
    print(v, w)

a b
b c
b d
c d
d a
d a
a b


In [16]:
for v, w, t in g.temporal_edges:
    print(v, w, t)

a b 1.0
b c 2.0
b d 2.0
c d 3.0
d a 4.0
d a 6.0
a b 7.0


## Reading Temporal Graphs from CSV files

In [8]:
t = pp.TemporalGraph.from_csv('ants_1_1.tedges')
print(t)

Temporal Graph with 89 nodes, 947 unique edges and 1911 events in [0.0, 1438.0]

Graph attributes
	src		<class 'torch.Tensor'> -> torch.Size([1911])
	t		<class 'torch.Tensor'> -> torch.Size([1911])
	dst		<class 'torch.Tensor'> -> torch.Size([1911])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'src', 't', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [11]:
m = pp.MultiOrderModel.from_temporal_graph(t, delta=60, max_order=3)
print(m.layers[1])
print(m.layers[2])
print(m.layers[3])

Directed graph with 89 nodes and 947 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([89, 1])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([947])

Graph attributes
	num_nodes		<class 'int'>

Directed graph with 947 nodes and 2654 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([947, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([2654])

Graph attributes
	num_nodes		<class 'int'>

Directed graph with 2654 nodes and 5561 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([2654, 3])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([5561])

Graph attributes
	num_nodes		<class 'int'>



In [14]:
t = pp.TemporalGraph.from_csv('sociopatterns_highschool_2013.tedges')
m = pp.MultiOrderModel.from_temporal_graph(t, delta=3600, max_order=3)
print(m.layers[1])
print(m.layers[2])
print(m.layers[3])

Directed graph with 327 nodes and 5818 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([327, 1])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([5818])

Graph attributes
	num_nodes		<class 'int'>

Directed graph with 5818 nodes and 21674 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([5818, 2])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([21674])

Graph attributes
	num_nodes		<class 'int'>

Directed graph with 21674 nodes and 37746 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([21674, 3])

Edge attributes
	edge_weight		<class 'torch.Tensor'> -> torch.Size([37746])

Graph attributes
	num_nodes		<class 'int'>



## Extracting Causal Topologies via Node-Time Event DAGs

We are often interested in the time respecting paths of a temporal graph.

A time respecting path is defined as a sequence of nodes $v_0,...,v_l$ where the corresponding edges occur in the right time ordering and with a maximum time difference of $\delta\in \N$. 

In order to extract those paths out of a temporal graph, we have to construct a time-unfolded directed acyclic graph (DAG) that represents the network and captures all causal structures.

The nodes of a DAG are node-time-events of the temporal graph, i.e a node `a-t` represents the node `a` at time `t`. 

Two nodes `a-t` and `b-t'` are connected (with exceptions, see code ) if `a-t` directly influences `b-t'` 


In [18]:
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
print(dag)

print(dag.mapping)
print(dag.data.edge_index)

pp.plot(dag, edge_color='lightgray')

Graph with 9 nodes and 7 edges

Node attributes
	node_name		<class 'list'>
	node_idx		<class 'list'>

Edge attributes
	edge_ts		<class 'torch.Tensor'> -> torch.Size([7])

Graph attributes
	num_nodes		<class 'int'>
	temporal_graph_index_map		<class 'list'>

a-1.0 -> 0
b-2.0 -> 1
c-3.0 -> 2
d-3.0 -> 3
d-4.0 -> 4
a-5.0 -> 5
d-6.0 -> 6
a-7.0 -> 7
b-8.0 -> 8

EdgeIndex([[0, 1, 1, 2, 4, 6, 7],
           [1, 2, 3, 4, 5, 7, 8]], sparse_size=(8, ?), nnz=7, sort_order=row)


With the following code, we can extract DAGs with only one single root node, which is not influenced by any other node-time event in the temporal graph.

In [19]:
x = pp.algorithms.extract_causal_trees(dag)
print(x)

{'a-1.0': tensor([[0, 1, 1, 2, 4],
        [1, 2, 3, 4, 5]], dtype=torch.int32), 'd-6.0': tensor([[6, 7],
        [7, 8]], dtype=torch.int32)}


## Higher-Order De Bruijn Graph Models for Causal Paths

With the DAG, we can now extract the time-respecting paths in our temporal graph.

In [20]:
paths = pp.DAGData.from_temporal_dag(dag)
print(paths.paths[0])
print(paths.paths[1])
print(g.mapping)

tensor([[0, 1, 1, 2, 4],
        [1, 2, 3, 4, 5]])
tensor([[6, 7],
        [7, 8]])
b -> 0
c -> 1
a -> 2
d -> 3



The path data allows us to construct a Higher-Order De Bruijn Graph model belong to our temporal network.

In [21]:
g2 = pp.HigherOrderGraph(paths, order=2, node_ids=g.mapping.node_ids)
pp.plot(g2)

In a nutshell, the following shows the steps that are needed to construct a Higher Order Graph. Here we construct our temporal graph from a `pyG TemporalData` by providing the sources, distances and timestamps.

In [22]:
# Create temporal network
data = TemporalData(src=torch.LongTensor([0,2,1,2,0,2,1,2]), dst=torch.LongTensor([2,3,2,4,2,3,2,4]), t=torch.LongTensor([1,2,3,4,5,6,7,8]))
g = pp.TemporalGraph(data, mapping=pp.IndexMap(['a', 'b', 'c', 'd', 'e']))

# Create event DAG and extract path data
dag = pp.algorithms.temporal_graph_to_event_dag(g, delta=1)
paths = pp.DAGData.from_temporal_dag(dag)

# Create Higher Order Graph
g2 = pp.HigherOrderGraph(paths, order=2)
pp.plot(g2)

We can also skip the step of creating an event DAG by just using the following code.

In [25]:
g2 = pp.HigherOrderGraph.from_temporal_graph(g, delta=1, order=2)
pp.plot(g2);